----
<div style="display: flex; align-items: center;">
    <img src="https://developers.lseg.com/content/dam/devportal/icons/logo/lseg-logo.svg" width="20%" style="vertical-align: top;">
    <h1 style="margin-left: 20px;">Data Library for Python</h1>
</div>

----

## Content - Pricing stream - How to send contributions
This notebook demonstrates how to use a Pricing stream to send contributions.

#### Learn more

To learn more about the LSEG Data Library for Python please join the LSEG Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [logging](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) into the LSEG Developer Community portal you will have free access to a number of learning materials on the [LSEG Developer Portal](https://developers.lseg.com/).

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [LSEG Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The LSEG Developer Community will be happy to help. 

## Set the configuration file location
For a better ease of use, you have the option to set initialization parameters of the LSEG Data Library in the _lseg-data.config.json_ configuration file. This file must be located beside your notebook, in your user folder or in a folder defined by the _LD_LIB_CONFIG_PATH_ environment variable. The _LD_LIB_CONFIG_PATH_ environment variable is the option used by this series of examples. The following code sets this environment variable.

In [10]:
import os
os.environ["LD_LIB_CONFIG_PATH"] = "../../../Configuration"

## Some Imports to start with

In [11]:
import lseg.data as rd
from lseg.data.content import pricing
from lseg.data.delivery.omm_stream import ContribType

## Open the data session

The open_session() function creates and open sessions based on the information contained in the lseg-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

Let's open our session. The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open. Note also the format expected in the config file here:

<img src="https://developers.lseg.com/content/dam/devportal/products-pages/workspace-upgrade/excel-related-com-api-upgrade/adxrtcontribute-dll/rtds%20contribute.PNG">

In this config file, you will see that we specified details for the highlighed `platform` & `rtds`; you ought to have already been provided with your contribution RTDS URL and DACS details. We can poit to these details in-code, in Python, in our code:

In [13]:
ld.open_session("platform.rtds")

<refinitiv.data.session.Definition object at 0x20e77c945e0 {name='rtds'}>

## Send contributions

### Define the service to contribute to

In [14]:
service = "ATS_GLOBAL_1" # Accept contribs on only a curated list of RICs

### Off-stream contributions

#### Helper function to display contrib responses

In [15]:
def display_contrib_response(response):
    if response.is_success:
        print(f">> Contribution successfully sent.")
    else:
        if response.nak_code:
            print(f'>> Contrib rejected because "{response.nak_message}".')
        if response.error:
            print(f'>> Contrib failed because "{response.error}\n\t{response.debug}"')


#### Send a contribution

In [16]:
response = ld.content.pricing.contribute(
    name = "TESTRIC01", 
    fields = {
        "BID": 5.555,
        "ASK": 4.444
    }, 
    service=service,
    contrib_type=ContribType.REFRESH
)

display_contrib_response(response)

>> Contribution successfully sent.


### On-stream contributions

#### Subscribe to contribution RICs

In [17]:
stream = ld.content.pricing.Definition(
    universe=["TESTRIC01", "TESTRIC02", "TESTRIC03"],
    service=service
).get_stream()

stream.open()

<OpenState.Opened: 'Opened'>

#### Get and display data

In [18]:
stream.get_snapshot()

Instrument  RDNDISPLAY        DSPLY_NAME        TIMACT TRADE_DATE  TRDTIM_1  \
0  TESTRIC01         123  BARCLAYS     LON  09:00:35.853 2023-12-05  15:39:00   
1  TESTRIC02         153  SANTANDER    HKG  08:55:51.696 2023-12-05  15:39:00   
2  TESTRIC03         123  SANTANDER    HKG  08:12:19.422 2023-12-05  10:12:00   

     BID    ASK  BIDSIZE  ASKSIZE  RECORDTYPE      ROW80_1  \
0  5.555  4.444       10       20         209  22-Mar-2023   
1    2.1   2.11       30        2         209  22-Mar-2023   
2    6.0    3.0       50       60         209    25JUL2021   

               ROW80_2    ROW80_3  STOCK_RIC SPARE_TS1 QUOTE_DATE  
0  contrib ask and bid          3  25JUL2021  16:30:00 2003-05-01  
1            25JUL2022  25JUL2023  25JUL2021  11:12:13 2022-07-27  
2            25JUL2022  25JUL2023  25JUL2021  11:12:13 2021-10-18

#### Send a contribution

In [19]:
response = stream.contribute(
    name = "TESTRIC02", 
    fields = {
        "BID": 2.10,
        "ASK": 2.11
    }
)

display_contrib_response(response)

>> Contribution successfully sent.


### Close the stream

In [20]:
stream.close()

<OpenState.Closed: 'Closed'>

## Close the session

In [21]:
ld.close_session()